In [91]:
import tensorflow as tf

In [92]:
import numpy as np
import pandas as pd

In [93]:
from tensorflow import keras

In [94]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [95]:
tf.__version__

'2.5.0'

In [96]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
training_set = train_datagen.flow_from_directory('new/train-com',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 767 images belonging to 2 classes.


In [97]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('new/test-com',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 330 images belonging to 2 classes.


In [98]:
cnn = tf.keras.models.Sequential()

In [99]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

In [100]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [101]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [102]:
cnn.add(tf.keras.layers.Flatten())

In [103]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [104]:
cnn.add(tf.keras.layers.Dense(units=200, activation='relu'))

In [105]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [106]:
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
opt = SGD(learning_rate=0.01)
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [107]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 20)

Epoch 1/20
24/24 [==============================] - 15s 511ms/step - loss: 0.6990 - accuracy: 0.5671 - val_loss: 0.6224 - val_accuracy: 0.6394
Epoch 2/20
24/24 [==============================] - 10s 397ms/step - loss: 0.5847 - accuracy: 0.6884 - val_loss: 0.6635 - val_accuracy: 0.6030
Epoch 3/20
24/24 [==============================] - 10s 398ms/step - loss: 0.5593 - accuracy: 0.7053 - val_loss: 0.5566 - val_accuracy: 0.6424
Epoch 4/20
24/24 [==============================] - 10s 428ms/step - loss: 0.4394 - accuracy: 0.8005 - val_loss: 0.6541 - val_accuracy: 0.6152
Epoch 5/20
24/24 [==============================] - 11s 448ms/step - loss: 0.4030 - accuracy: 0.8123 - val_loss: 0.4759 - val_accuracy: 0.7576
Epoch 6/20
24/24 [==============================] - 12s 498ms/step - loss: 0.2961 - accuracy: 0.8787 - val_loss: 0.4459 - val_accuracy: 0.8000
Epoch 7/20
24/24 [==============================] - 11s 457ms/step - loss: 0.2369 - accuracy: 0.9048 - val_loss: 0.6085 - val_accuracy: 0.7364

In [109]:
import numpy as np
from keras.preprocessing import image
n,b,m=0,0,0
for  i in range(122,247):
    
    path="dataset/test/ntest/Normal case ("+ str(i) +").jpg"
    test_image = image.load_img(path, target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = cnn.predict(test_image)
    result[0][0]=int(result[0][0])
    result[0][1]=int(result[0][1])
    result[0][2]=int(result[0][2])
    if(result[0][1]==1):
        m+=1
    elif(result[0][2]==1):
        n+=1
    else:
        b+=1
print(training_set.class_indices)
print(b,m,n)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [110]:
from tensorflow.keras.models import model_from_json

In [114]:
model_json = cnn.to_json()
with open("cnn.json", "w") as json_file:
    json_file.write(model_json)
cnn.save_weights("cnn.h5")

In [115]:
json_file = open('cnn.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("cnn.h5")

In [116]:
import numpy as np
from keras.preprocessing import image
n,b,m=0,0,0
for  i in range(122,247):
    
    path="dataset/test/ntest/Normal case ("+ str(i) +").jpg"
    test_image = image.load_img(path, target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = loaded_model.predict(test_image)
    result[0][0]=int(result[0][0])
    result[0][1]=int(result[0][1])
    result[0][2]=int(result[0][2])
    if(result[0][1]==1):
        m+=1
    elif(result[0][2]==1):
        n+=1
    else:
        b+=1
print(training_set.class_indices)
print(b,m,n)

IndexError: index 1 is out of bounds for axis 0 with size 1